In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')

Libraries imported.


In [2]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [3]:
#https://www.data.gouv.fr/en/datasets/quartiers-administratifs/
path = 'quartier_paris.csv'
df= pd.read_csv(path)
df.head()


,N_SQ_QU,C_QU,C_QUINSEE,L_QU,C_AR,N_SQ_AR,PERIMETRE,SURFACE,Geometry X Y,Geometry
0,750000008,8,7510204,Bonne-Nouvelle,2,750000002,2233.976030,2.814482e+05,48.867150,2.350080
1,750000018,18,7510502,Jardin-des-Plantes,5,750000005,4052.729521,7.983894e+05,48.841940,2.356894
2,750000030,30,7510802,Faubourg-du-Roule,8,750000008,3773.673073,7.965891e+05,48.874136,2.304119
3,750000052,52,7511304,Croulebarbe,13,750000013,3289.230480,6.920677e+05,48.833734,2.347673
4,750000061,61,7511601,Auteuil,16,750000016,12452.253930,6.383888e+06,48.850622,2.252277


In [4]:
df.rename(columns={'Geometry X Y': 'Latitude', 'Geometry': 'Longitude', 'L_QU': 'Neighborhood'}, inplace=True)


In [5]:
from pandas.io.html import read_html
page='https://en.wikipedia.org/wiki/Quarters_of_Paris'
pop=pd.read_html(page, attrs={"class": "wikitable"})
pop[0].head()

,Arrondissement(Districts),Quartiers(Quarters),Quartiers(Quarters).1,Population in1999[3],Area(hectares)[3],Map
0,"1st arrondissement(Called ""du Louvre"")",1st,Saint-Germain-l'Auxerrois,1672,86.9,NaN
1,"1st arrondissement(Called ""du Louvre"")",2nd,Les Halles,8984,41.2,NaN
2,"1st arrondissement(Called ""du Louvre"")",3rd,Palais-Royal,3195,27.4,NaN
3,"1st arrondissement(Called ""du Louvre"")",4th,Place-Vendôme,3044,26.9,NaN
4,"2nd arrondissement(Called ""de la Bourse"")",5th,Gaillon,1345,18.8,NaN


In [6]:
density=pop[0]

In [7]:
density.rename(columns={'Quartiers(Quarters).1': 'Neighborhood', 'Arrondissement(Districts)': 'Borough', 'Population in1999[3]': 'Population', 'Quartiers(Quarters)': 'C_QU' }, inplace=True)
density.head()

,Borough,C_QU,Neighborhood,Population,Area(hectares)[3],Map
0,"1st arrondissement(Called ""du Louvre"")",1st,Saint-Germain-l'Auxerrois,1672,86.9,NaN
1,"1st arrondissement(Called ""du Louvre"")",2nd,Les Halles,8984,41.2,NaN
2,"1st arrondissement(Called ""du Louvre"")",3rd,Palais-Royal,3195,27.4,NaN
3,"1st arrondissement(Called ""du Louvre"")",4th,Place-Vendôme,3044,26.9,NaN
4,"2nd arrondissement(Called ""de la Bourse"")",5th,Gaillon,1345,18.8,NaN


In [8]:
density.replace(['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11th', '12th', '13th', '14th', '15th', '16th', '17th', '18th', '19th', '20th',
                 '21st', '22nd', '23rd', '24th','25th', '26th', '27th', '28th', '29th', '30th', '31st', '32nd', '33rd', '34th', '35th', '36th', '37th', '38th', '39th', 
                 '40th', '41st', '42nd', '43rd', '44th', '45th', '46th', '47th', '48th', '49th', '50th', '51st', '52nd', '53rd','54th', '55th', '56th', '57th', '58th',
                 '59th', '60th', '61st', '62nd', '63rd', '64th', '65th', '66th', '67th', '68th', '69th', '70th', '71st', '72nd', '73rd', '74th', '75th', '76th', '77th',
                 '78th', '79th', '80th' ],
                 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24','25', '26', 
                '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', 
                  '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71','72','73','74','75','76','77',
                  '78','79', '80'], inplace=True)
density.head()

,Borough,C_QU,Neighborhood,Population,Area(hectares)[3],Map
0,"1st arrondissement(Called ""du Louvre"")",1,Saint-Germain-l'Auxerrois,1672,86.9,NaN
1,"1st arrondissement(Called ""du Louvre"")",2,Les Halles,8984,41.2,NaN
2,"1st arrondissement(Called ""du Louvre"")",3,Palais-Royal,3195,27.4,NaN
3,"1st arrondissement(Called ""du Louvre"")",4,Place-Vendôme,3044,26.9,NaN
4,"2nd arrondissement(Called ""de la Bourse"")",5,Gaillon,1345,18.8,NaN


In [9]:
density['C_QU']=density['C_QU'].astype(int)

In [10]:
paris=pd.merge(df, density, how='left', on= 'C_QU')
paris.head()

,N_SQ_QU,C_QU,C_QUINSEE,Neighborhood_x,C_AR,N_SQ_AR,PERIMETRE,SURFACE,Latitude,Longitude,Borough,Neighborhood_y,Population,Area(hectares)[3],Map
0,750000008,8,7510204,Bonne-Nouvelle,2,750000002,2233.976030,2.814482e+05,48.867150,2.350080,"2nd arrondissement(Called ""de la Bourse"")",Bonne-Nouvelle,9595,28.2,NaN
1,750000018,18,7510502,Jardin-des-Plantes,5,750000005,4052.729521,7.983894e+05,48.841940,2.356894,"5th arrondissement(Called ""du Panthéon"")",Jardin-des-Plantes,18005,79.8,NaN
2,750000030,30,7510802,Faubourg-du-Roule,8,750000008,3773.673073,7.965891e+05,48.874136,2.304119,"8th arrondissement(Called ""de l'Élysée"")",Faubourg-du-Roule,10038,79.6,NaN
3,750000052,52,7511304,Croulebarbe,13,750000013,3289.230480,6.920677e+05,48.833734,2.347673,"13th arrondissement(Called ""des Gobelins"")",Croulebarbe,19526,69.2,NaN
4,750000061,61,7511601,Auteuil,16,750000016,12452.253930,6.383888e+06,48.850622,2.252277,"16th arrondissement(Called ""de Passy"")",Auteuil,67967,303.0,NaN


In [11]:
paris.rename(columns={'Neighborhood_x': 'Neighborhood'}, inplace=True)
paris.head()

,N_SQ_QU,C_QU,C_QUINSEE,Neighborhood,C_AR,N_SQ_AR,PERIMETRE,SURFACE,Latitude,Longitude,Borough,Neighborhood_y,Population,Area(hectares)[3],Map
0,750000008,8,7510204,Bonne-Nouvelle,2,750000002,2233.976030,2.814482e+05,48.867150,2.350080,"2nd arrondissement(Called ""de la Bourse"")",Bonne-Nouvelle,9595,28.2,NaN
1,750000018,18,7510502,Jardin-des-Plantes,5,750000005,4052.729521,7.983894e+05,48.841940,2.356894,"5th arrondissement(Called ""du Panthéon"")",Jardin-des-Plantes,18005,79.8,NaN
2,750000030,30,7510802,Faubourg-du-Roule,8,750000008,3773.673073,7.965891e+05,48.874136,2.304119,"8th arrondissement(Called ""de l'Élysée"")",Faubourg-du-Roule,10038,79.6,NaN
3,750000052,52,7511304,Croulebarbe,13,750000013,3289.230480,6.920677e+05,48.833734,2.347673,"13th arrondissement(Called ""des Gobelins"")",Croulebarbe,19526,69.2,NaN
4,750000061,61,7511601,Auteuil,16,750000016,12452.253930,6.383888e+06,48.850622,2.252277,"16th arrondissement(Called ""de Passy"")",Auteuil,67967,303.0,NaN


In [12]:
paris.drop(['C_QUINSEE','C_AR','PERIMETRE','SURFACE','Area(hectares)[3]', 'Map', 'Neighborhood_y'], axis=1, inplace=True)
paris.head(2)

,N_SQ_QU,C_QU,Neighborhood,N_SQ_AR,Latitude,Longitude,Borough,Population
0,750000008,8,Bonne-Nouvelle,750000002,48.86715,2.350080,"2nd arrondissement(Called ""de la Bourse"")",9595
1,750000018,18,Jardin-des-Plantes,750000005,48.84194,2.356894,"5th arrondissement(Called ""du Panthéon"")",18005


In [13]:
paris=paris[['N_SQ_AR','Borough', 'N_SQ_QU','C_QU','Neighborhood', 'Latitude', 'Longitude', 'Population' ]]
paris.head()

,N_SQ_AR,Borough,N_SQ_QU,C_QU,Neighborhood,Latitude,Longitude,Population
0,750000002,"2nd arrondissement(Called ""de la Bourse"")",750000008,8,Bonne-Nouvelle,48.867150,2.350080,9595
1,750000005,"5th arrondissement(Called ""du Panthéon"")",750000018,18,Jardin-des-Plantes,48.841940,2.356894,18005
2,750000008,"8th arrondissement(Called ""de l'Élysée"")",750000030,30,Faubourg-du-Roule,48.874136,2.304119,10038
3,750000013,"13th arrondissement(Called ""des Gobelins"")",750000052,52,Croulebarbe,48.833734,2.347673,19526
4,750000016,"16th arrondissement(Called ""de Passy"")",750000061,61,Auteuil,48.850622,2.252277,67967


In [14]:
# @hidden_cell
CLIENT_ID = '50BHQPYTUIFSJ5OKIXEM4UYUK10O0ZFB0GN3U21G5ULP23GI' # your Foursquare ID
CLIENT_SECRET = 'MFFCNYELMFGEDVPGCNAYTAXOIOMGKV145OPUWHBXKLFQVGBC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
paris_venues = getNearbyVenues(names=paris['Neighborhood'],
                                   latitudes=paris['Latitude'],
                                   longitudes=paris['Longitude']
                                  )

Bonne-Nouvelle
Jardin-des-Plantes
Faubourg-du-Roule
Croulebarbe
Auteuil
Saint-Merri
Notre-Dame
Monnaie
Champs-Elysées
Saint-Vincent-de-Paul
Hôpital-Saint-Louis
Sainte-Marguerite
Bercy
Amérique
Belleville
Enfants-Rouges
Odéon
Notre-Dame-des-Champs
Salpêtrière
Epinettes
Grandes-Carrières
Roquette
Archives
Invalides
Ecole-Militaire
Combat
Montparnasse
Bel-Air
Gare
Pont-de-Flandre
Plaine de Monceaux
Sainte-Avoie
Rochechouart
Folie-Méricourt
Saint-Fargeau
Batignolles
Saint-Thomas-d'Aquin
Faubourg-Montmartre
Palais-Royal
Clignancourt
Saint-Victor
Goutte-d'Or
Place-Vendôme
La Chapelle
Javel
Sorbonne
Porte-Saint-Denis
Petit-Montrouge
Saint-Lambert
Père-Lachaise
Muette
Ternes
Vivienne
Gros-Caillou
Parc-de-Montsouris
Mail
Arts-et-Métiers
Saint-Georges
Picpus
Grenelle
Saint-Germain-l'Auxerrois
Gaillon
Val-de-Grâce
Europe
Plaisance
Porte-Dauphine
Saint-Germain-des-Prés
Porte-Saint-Martin
Maison-Blanche
Halles
Chaillot
Saint-Gervais
Chaussée-d'Antin
Saint-Ambroise
Necker
Arsenal
Madeleine
Quinze-Vi

In [17]:
paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_onehot['Neighborhood'] = paris_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cemetery,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Lyonese Bouchon,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Provençal Restaurant,Pub,Racecourse,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Resort,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szech

In [18]:
paris_onehot['gymall'] = paris_onehot.loc[:,['Gym','Gym / Fitness Center']].sum(axis=1)

In [19]:
paris_grouped =paris_onehot.groupby('Neighborhood').mean().reset_index()

paris_grouped.head()


,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cemetery,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Lyonese Bouchon,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Provençal Restaurant,Pub,Racecourse,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Resort,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szech

In [20]:
num_top_venues = 5

for hood in paris_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = paris_grouped[paris_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Amérique----
               venue  freq
0        Supermarket  0.14
1               Café  0.14
2  French Restaurant  0.14
3               Pool  0.07
4             Bistro  0.07


----Archives----
                 venue  freq
0       Clothing Store  0.09
1    French Restaurant  0.07
2          Art Gallery  0.06
3         Burger Joint  0.04
4  Japanese Restaurant  0.04


----Arsenal----
               venue  freq
0  French Restaurant  0.19
1              Hotel  0.05
2              Plaza  0.04
3   Tapas Restaurant  0.04
4          Gastropub  0.04


----Arts-et-Métiers----
                venue  freq
0   French Restaurant  0.11
1               Hotel  0.07
2        Cocktail Bar  0.05
3  Italian Restaurant  0.05
4         Coffee Shop  0.04


----Auteuil----
                 venue  freq
0         Tennis Court  0.31
1              Stadium  0.19
2               Office  0.06
3  Sporting Goods Shop  0.06
4     Botanical Garden  0.06


----Batignolles----
                venue  freq
0   French R

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = paris_grouped['Neighborhood']

for ind in np.arange(paris_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amérique,French Restaurant,Café,Supermarket,Bistro,Street Art,Bakery,Bed & Breakfast,Tram Station,Park,Plaza
1,Archives,Clothing Store,French Restaurant,Art Gallery,Japanese Restaurant,Burger Joint,Coffee Shop,Café,Tea Room,Bookstore,Bakery
2,Arsenal,French Restaurant,Hotel,Plaza,Park,Italian Restaurant,Tapas Restaurant,Seafood Restaurant,Gastropub,Bakery,Pedestrian Plaza
3,Arts-et-Métiers,French Restaurant,Hotel,Italian Restaurant,Cocktail Bar,Bar,Chinese Restaurant,Wine Bar,Vietnamese Restaurant,Coffee Shop,Restaurant
4,Auteuil,Tennis Court,Stadium,Sporting Goods Shop,French Restaurant,Botanical Garden,Garden,Office,Outdoors & Recreation,Museum,Racecourse
5,Batignolles,French Restaurant,Hotel,Italian Restaurant,Bar,Bistro,Café,Restaurant,Japanese Restaurant,Pizza Place,Park
6,Bel-Air,Recreation Center,Plaza,French Restaurant,Café,Playground,Sports Club,Cupcake Shop,Ethiopian Restaurant,Cosmetics Shop,Fountain
7,Belleville,Bar,French Restaurant,Café,Italian Restaurant,Theater,Cocktail Bar,Restaurant,Chinese Restaurant,Park,Diner
8,Bercy,Hotel,French Restaurant,Bakery,Italian Restaurant,Wine Bar,Sushi Restaurant,Bar,Bistro,Museum,Supermarket
9,Bonne-Nouvelle,Cocktail Bar,Hotel,Bakery,French Restaurant,Italian Restaurant,Bar,Wine Bar,Coffee Shop,gymall,Thai Restaurant


In [23]:
paris_gym=paris_grouped[['Neighborhood', 'gymall']]
paris_gym.head()

,Neighborhood,gymall
0,Amérique,0.000000
1,Archives,0.000000
2,Arsenal,0.012658
3,Arts-et-Métiers,0.000000
4,Auteuil,0.000000


In [24]:
paris_all=pd.merge(paris, paris_gym, how='left', on='Neighborhood')
paris_all.head()

,N_SQ_AR,Borough,N_SQ_QU,C_QU,Neighborhood,Latitude,Longitude,Population,gymall
0,750000002,"2nd arrondissement(Called ""de la Bourse"")",750000008,8,Bonne-Nouvelle,48.867150,2.350080,9595,0.020000
1,750000005,"5th arrondissement(Called ""du Panthéon"")",750000018,18,Jardin-des-Plantes,48.841940,2.356894,18005,0.014286
2,750000008,"8th arrondissement(Called ""de l'Élysée"")",750000030,30,Faubourg-du-Roule,48.874136,2.304119,10038,0.010000
3,750000013,"13th arrondissement(Called ""des Gobelins"")",750000052,52,Croulebarbe,48.833734,2.347673,19526,0.000000
4,750000016,"16th arrondissement(Called ""de Passy"")",750000061,61,Auteuil,48.850622,2.252277,67967,0.000000


In [25]:
paris_all.rename(columns={'gymall': 'Gym'}, inplace=True)

In [26]:
paris_all.describe()

,N_SQ_AR,N_SQ_QU,C_QU,Latitude,Longitude,Population,Gym
count,8.000000e+01,8.000000e+01,80.0000,80.000000,80.000000,80.000000,80.000000
mean,7.500000e+08,7.500000e+08,40.5000,48.860991,2.343915,26573.137500,0.013354
std,5.802662e+00,2.323790e+01,23.2379,0.018429,0.035163,20235.804847,0.021321
min,7.500000e+08,7.500000e+08,1.0000,48.823128,2.252277,1345.000000,0.000000
25%,7.500000e+08,7.500000e+08,20.7500,48.848734,2.324470,9413.750000,0.000000
50%,7.500000e+08,7.500000e+08,40.5000,48.862451,2.344880,21418.500000,0.010000
75%,7.500000e+08,7.500001e+08,60.2500,48.873565,2.363586,39025.250000,0.020000
max,7.500000e+08,7.500001e+08,80.0000,48.895556,2.433178,82032.000000,0.125000


In [27]:
address = 'paris'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [28]:
from folium.plugins import HeatMap

In [29]:
map_paris = folium.Map(location=[latitude, longitude], zoom_start=10)
max_amount = float(paris_all['Gym'].max())
for lat, lng, borough, neighbourhood, gym in zip(paris_all['Latitude'], paris_all['Longitude'], paris_all['Borough'], paris_all['Neighborhood'], paris_all['Gym']):
    label = '{}, {}, {}'.format(neighbourhood, borough, gym)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.2,
        parse_html=False).add_to(map_paris)  
    


hm_wide = HeatMap( list(zip(paris_all.Latitude.values, paris_all.Longitude.values, paris_all.Gym.values)),
                   min_opacity=0.3,
                   
                   radius=20, blur=15, 
                   max_zoom=10, 
                   max_val= max_amount
                 )


map_paris.add_child(hm_wide)

In [30]:
map_paris1 = folium.Map(location=[latitude, longitude], zoom_start=10)
max_amount = float(paris_all['Population'].max())
for lat, lng, borough, neighbourhood, pop in zip(paris_all['Latitude'], paris_all['Longitude'], paris_all['Borough'], paris_all['Neighborhood'], paris_all['Population']):
    label = '{}, {}, {}'.format(neighbourhood, borough, pop)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris1)  
    


hm_wide = HeatMap( list(zip(paris_all.Latitude.values, paris_all.Longitude.values, paris_all.Population.values)),
                   min_opacity=0.2,
                   
                   radius=20, blur=15, 
                   max_zoom=10, 
                   max_val= max_amount
                 )


map_paris1.add_child(hm_wide)

In [32]:
paris_all_new=paris_all.drop(paris_all[(paris_all.Population< 26600) | (paris_all.Gym> 0.013)].index)
paris_all_new

,N_SQ_AR,Borough,N_SQ_QU,C_QU,Neighborhood,Latitude,Longitude,Population,Gym
4,750000016,"16th arrondissement(Called ""de Passy"")",750000061,61,Auteuil,48.850622,2.252277,67967,0.000000
10,750000010,"10th arrondissement(Called ""de l'Entrepôt"")",750000040,40,Hôpital-Saint-Louis,48.876008,2.368123,29870,0.000000
13,750000019,"19th arrondissement(Called ""des Buttes-Chaumont"")",750000075,75,Amérique,48.881638,2.395440,55365,0.000000
14,750000020,"20th arrondissement(Called ""de Ménilmontant"")",750000077,77,Belleville,48.871531,2.387549,35773,0.000000
27,750000012,"12th arrondissement(Called ""de Reuilly"")",750000045,45,Bel-Air,48.837996,2.433178,33976,0.000000
28,750000013,"13th arrondissement(Called ""des Gobelins"")",750000050,50,Gare,48.827527,2.372398,69008,0.000000
30,750000017,"17th arrondissement(Called ""des Batignolles-Mo...",750000066,66,Plaine de Monceaux,48.885044,2.302910,38958,0.000000
34,750000020,"20th arrondissement(Called ""de Ménilmontant"")",750000078,78,Saint-Fargeau,48.871035,2.406172,42087,0.000000
35,750000017,"17th arrondissement(Called ""des Batignolles-Mo...",750000067,67,Batignolles,48.888482,2.313856,38691,0.010638
39,750000018,"18th arrondissement(Called ""des Buttes-Montmar...",750000070,70,Clignancourt,48.891668,2.345979,64868,0.000000


In [41]:
paris_all_new.sort_values(["Population"], ascending=False, inplace=True)
paris_all_new

,N_SQ_AR,Borough,N_SQ_QU,C_QU,Neighborhood,Latitude,Longitude,Population,Gym
28,750000013,"13th arrondissement(Called ""des Gobelins"")",750000050,50,Gare,48.827527,2.372398,69008,0.000000
4,750000016,"16th arrondissement(Called ""de Passy"")",750000061,61,Auteuil,48.850622,2.252277,67967,0.000000
39,750000018,"18th arrondissement(Called ""des Buttes-Montmar...",750000070,70,Clignancourt,48.891668,2.345979,64868,0.000000
68,750000013,"13th arrondissement(Called ""des Gobelins"")",750000051,51,Maison-Blanche,48.823128,2.352433,64797,0.000000
58,750000012,"12th arrondissement(Called ""de Reuilly"")",750000046,46,Picpus,48.830359,2.428827,62947,0.000000
13,750000019,"19th arrondissement(Called ""des Buttes-Chaumont"")",750000075,75,Amérique,48.881638,2.395440,55365,0.000000
79,750000019,"19th arrondissement(Called ""des Buttes-Chaumont"")",750000073,73,Villette,48.887661,2.374468,53650,0.000000
44,750000015,"15th arrondissement(Called ""de Vaugirard"")",750000060,60,Javel,48.839060,2.278076,49092,0.000000
49,750000020,"20th arrondissement(Called ""de Ménilmontant"")",750000079,79,Père-Lachaise,48.863719,2.395273,42332,0.000000
34,750000020,"20th arrondissement(Called ""de Ménilmontant"")",750000078,78,Saint-Fargeau,48.871035,2.406172,42087,0.000000


In [36]:
map_paris = folium.Map(location=[latitude, longitude], zoom_start=10)
max_amount = float(paris_all_new['Population'].max())
for lat, lng, borough, neighbourhood, gym, pop in zip(paris_all_new['Latitude'], paris_all_new['Longitude'], paris_all_new['Borough'], paris_all_new['Neighborhood'], paris_all_new['Gym'], paris_all_new['Population']):
    label = '{}, {}, {},{}'.format(neighbourhood, borough, gym, pop)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.2,
        parse_html=False).add_to(map_paris)  
    


hm_wide = HeatMap( list(zip(paris_all_new.Latitude.values, paris_all_new.Longitude.values, paris_all_new.Population.values)),
                   min_opacity=0.3,
                   radius=20, blur=15, 
                   max_zoom=10, 
                   max_val= max_amount
                 )


map_paris.add_child(hm_wide)